In [23]:
import json
import pandas as pd
import glob

monolingual_files = glob.glob("nbs/final_eval" + '/**/monolingual/**/*predictions.json', recursive=True)
crosslingual_files = glob.glob("nbs/final_eval" + '/**/crosslingual/**/*predictions.json', recursive=True)
d_tasks = json.load(open("data/splits/tasks.json"))
mono_gs = json.load(open("nbs/final_eval/mono_dev_gs.json"))
cross_gs = json.load(open("nbs/final_eval/cross_dev_gs.json"))

cross_preds = "official/contrastive/snowflake_mv2/crosslingual/20250131-000403/crosslingual_predictions.json"

def get_gs(d_task, task_name, d_gs):
    
    if task_name == "crosslingual":
        df_gs = pd.DataFrame({"cross": d_tasks["crosslingual"]})
    else:
        df_gs = pd.DataFrame(d_task[task_name])
        
    df_gs = df_gs.loc["posts_dev", :].reset_index().explode("posts_dev").rename(columns={"index": "lang", "posts_dev": "post_id"}).astype(str)
    df_gs["gs"] = df_gs["post_id"].map(d_gs)
    df_gs = df_gs.explode("gs")
    return df_gs

def get_preds(file):
    return pd.DataFrame(json.load(open(file))).T.apply(list, axis=1).explode().reset_index().rename(columns={"index": "post_id", 0: "pred"})

df_gs_cross = get_gs(d_tasks, "crosslingual", cross_gs)

df_cross_preds = get_preds(cross_preds)

In [28]:
df_cross

,lang,post_id,gs,pred,correct
0,cross,34,77464,6399,False
1,cross,34,77464,77464,True
2,cross,34,77464,77462,False
3,cross,34,77464,116777,False
4,cross,34,77464,73564,False
...,...,...,...,...,...
6505,cross,28059,64893,43041,False
6506,cross,28059,64893,40361,False
6507,cross,28059,64893,153751,False
6508,cross,28059,64893,139255,False


In [33]:
df_cross = df_gs_cross.merge(df_cross_preds, on="post_id")
df_cross["correct"] = df_cross["gs"] == df_cross["pred"]
ls_check_inc = df_cross.groupby("post_id")["correct"].any()
ls_check_inc = ls_check_inc[~ls_check_inc].index
df_check_inc = df_cross[df_cross["post_id"].isin(ls_check_inc)]
df_check_inc

,lang,post_id,gs,pred,correct
20,cross,116,106143,101502,False
21,cross,116,106143,104228,False
22,cross,116,106143,68966,False
23,cross,116,106143,167778,False
24,cross,116,106143,104307,False
...,...,...,...,...,...
6375,cross,27558,86293,91030,False
6376,cross,27558,86293,150274,False
6377,cross,27558,86293,90987,False
6378,cross,27558,86293,141885,False


In [ ]:
from src.datasets import TextConcatPosts, TextConcatFactCheck
from src import config
posts_path = config.POSTS_PATH
tasks_path = config.TASKS_PATH
task_name = "crosslingual"
fact_checks_path = config.FACT_CHECKS_PATH

posts = TextConcatPosts(posts_path, tasks_path, task_name=task_name, gs_path=gs_path, lang=lang)
fact_checks = TextConcatFactCheck(fact_checks_path, tasks_path, task_name=task_name, lang=lang)
